## Leading Eigenvector Dynamics Analysis (LEiDA) framework to fMRI data

In [1]:
import os
import numpy as np
import pandas as pd
import csv

In [2]:
MAIN_PATH = '/Volumes/FedericaCardillo/pre-processing/projects/PROJECT_EGRET-AAA' # Project directory 
bold_dir = os.path.join(MAIN_PATH, 'derivatives', 'pRFM') # BOLD file directory 
freesurfer_dir = os.path.join(MAIN_PATH, 'derivatives', 'freesurfer') # Freesurfer labels directory 
timeseries_output = os.path.join(MAIN_PATH, 'data', 'time_series')  # Output time series
roi_output = os.path.join(MAIN_PATH, 'data', 'rois_coordinates.csv') # Output roi coordinates 

subjects = [f"sub-{i:02}" for i in range(2, 47)]
sessions = ['ses-01', 'ses-02']
rois = ['V1', 'V2', 'V3', 'V4', 'LO']
hemis = ['lh', 'rh']
roi_files = []
for hemi in hemis:
    for roi in rois:
        label_filename = f"{hemi}.manual_{roi}.label"
        label_name = f"{hemi}.{roi}"
        roi_files.append((label_filename, label_name))

In [3]:
"""Load and read the Freesurfer label to compute the center vertex as the average of x, y, z coordinates."""
def label_center(filepath):
    coordinates = [] # Empty list to store the coordinates

    with open(filepath, 'r') as f: # Open label in reading mode 
        lines = f.readlines() # Read all lines from the file into a list

    for line in lines[2:]: # Itearte over all the lines ignoring the first two
        parts = line.split() # Split the line into parts
        # Get x, y, z coordinates (second, third, fourth values)
        x = float(parts[1])
        y = float(parts[2])
        z = float(parts[3])
        coordinates.append([x, y, z]) # Add the coordinates to the list

    coordinates = np.array(coordinates)
    center = coordinates.mean(axis=0) # # Calculate the mean x, y, z position
    return center

"""Get the FreeSurfer label to extract a list vertex indices."""
def load_label_indices(label_path):
    indices = [] # Empty list to store vertex indices
    with open(label_path, 'r') as f:
        lines = f.readlines()

    # Skip the first two lines (header)
    for line in lines[2:]:
        parts = line.split()
        index = int(parts[0]) # Get the index from the first value in each line
        indices.append(index) # Add the vertex index to the list

    return indices

In [ ]:
# EXtract the roi coordinates for a representative subject 
results = []  # Empty list to store the results: (label, x, y, z)
for filename, label in roi_files:
    label_path = os.path.join(freesurfer_dir, 'sub-46', 'label', filename) # Path to representative subject
    center = label_center(label_path) # Compute the center as the average x, y, z
    results.append((label, *center)) # Add the label name and its center

# Save all center to a CSV file
with open(roi_output, 'w', newline='') as f:
    csv.writer(f).writerows(results)

# Extract the time series for each subject
for subject in subjects:
    session_path = None  # Track which session is used
    bold_path = None     # Full path to the BOLD time series file
    for ses in sessions: # Try to find the BOLD file in either ses-01 or ses-02
        temp_path = os.path.join(bold_dir, subject, ses, 'nordic', f'{subject}_{ses}_task-RestingState_hemi-LR_desc-avg_bold_GM.npy')
        if os.path.exists(temp_path):
            session_path = ses
            bold_path = temp_path
            break 
    if bold_path is None: # Skip the subject if no BOLD file was found
        print(f'No BOLD file for {subject}')
        continue
    bold_data = np.load(bold_path).T # Load the BOLD data with vertices x timepoints
    label_dir = os.path.join(freesurfer_dir, subject, 'label') # Label directory 

    # Collect all vertex indices from ROI label files
    #all_indices = []
    #for roi in rois: 
    #    for hemi in ['lh', 'rh']:
    #        label_path = os.path.join(label_dir, f'{hemi}.manual_{roi}.label')
    #        indices = load_label_indices(label_path)
    #        all_indices.append(indices)

    #all_indices = np.concatenate(all_indices) # Combine vertex indices
    #selected_data = bold_data[all_indices, :] # Get the time series

    # Save the ouput 
    #output_dir = os.path.join(timeseries_output, subject) # Subject specific directory 
    #os.makedirs(output_dir, exist_ok=True)
    #output_path = os.path.join(output_dir, f'{subject}.csv')
    #with open(output_path, 'w', newline='') as f:
    #    csv.writer(f).writerows(selected_data)

    # Compute average time series for each ROI and store in a list
    averaged_timeseries = []

    for roi in rois: 
        for hemi in ['lh', 'rh']:
            label_name = f'{hemi}.{roi}'
            label_path = os.path.join(label_dir, f'{hemi}.manual_{roi}.label')
            indices = load_label_indices(label_path)

            if not indices:
                print(f"No indices found for {label_name} in {subject}.")
                continue

            roi_timeseries = bold_data[indices, :]           # shape: [num_vertices_in_roi x timepoints]
            mean_timeseries = 
            [timepoints]
            averaged_timeseries.append(mean_timeseries)

    # Convert to array: [N_ROIs x N_timepoints]
    averaged_timeseries = np.array(averaged_timeseries)

    # Save the output
    output_dir = os.path.join(timeseries_output, subject)
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f'{subject}.csv')
    np.savetxt(output_path, averaged_timeseries, delimiter=',')

No BOLD file for sub-03
No BOLD file for sub-13
No BOLD file for sub-14
No BOLD file for sub-15
No BOLD file for sub-16
No BOLD file for sub-17
No BOLD file for sub-18
No BOLD file for sub-20
No BOLD file for sub-24
No BOLD file for sub-26
No BOLD file for sub-37
No BOLD file for sub-38
No BOLD file for sub-39
No BOLD file for sub-40
No BOLD file for sub-41
No BOLD file for sub-42
No BOLD file for sub-43
No BOLD file for sub-44
No BOLD file for sub-45
No BOLD file for sub-46


In [ ]:

    # Collect all vertex indices from ROI label files
    all_indices = []
    for roi in rois: 
        for hemi in ['lh', 'rh']:
            label_path = os.path.join(label_dir, f'{hemi}.manual_{roi}.label')
            indices = load_label_indices(label_path)
            all_indices.append(indices)

    all_indices = np.concatenate(all_indices) # Combine vertex indices
    selected_data = bold_data[all_indices, :] # Get the time series

    # Save the ouput 
    output_dir = os.path.join(timeseries_output, subject) # Subject specific directory 
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f'{subject}.csv')
    with open(output_path, 'w', newline='') as f:
        csv.writer(f).writerows(selected_data)


⸻

➕ Replace With This:
